## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-11-15-52-07 +0000,bbc,South Sudan vice-president charged with murder...,https://www.bbc.com/news/articles/c07vkln2ezro...
1,2025-09-11-15-51-33 +0000,nyt,"Where and When Charlie Kirk Was Shot: Maps, Vi...",https://www.nytimes.com/2025/09/10/us/charlie-...
2,2025-09-11-15-46-43 +0000,bbc,Breakthrough in hunt for 'college age' suspect...,https://www.bbc.com/news/articles/c5yke6drl42o...
3,2025-09-11-15-45-16 +0000,nyt,Did a Brooklyn Couple Kill a Neighbor’s Trees ...,https://www.nytimes.com/2025/09/11/us/trees-po...
4,2025-09-11-15-44-07 +0000,wapo,Russian diva condemns Putin’s war on Ukraine i...,https://www.washingtonpost.com/world/2025/09/1...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,kirk,71
9,charlie,65
76,trump,38
21,shooting,21
305,new,21


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
223,2025-09-11-00-08-00 +0000,wsj,Charlie Kirk was a provocative conservative in...,https://www.wsj.com/politics/charlie-kirk-turn...,236
289,2025-09-10-20-36-13 +0000,nyt,Trump and Leaders Across Political Spectrum Re...,https://www.nytimes.com/2025/09/10/us/charlie-...,218
156,2025-09-11-09-00-00 +0000,wsj,The killing of the conservative activist Charl...,https://www.wsj.com/politics/political-violenc...,216
92,2025-09-11-12-11-58 +0000,latimes,Charlie Kirk shooting reignites campus free sp...,https://www.latimes.com/california/story/2025-...,207
333,2025-09-10-19-03-28 +0000,latimes,Shooting of Charlie Kirk was 'political assass...,https://www.latimes.com/california/story/2025-...,206


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
223,236,2025-09-11-00-08-00 +0000,wsj,Charlie Kirk was a provocative conservative in...,https://www.wsj.com/politics/charlie-kirk-turn...
232,80,2025-09-10-23-40-09 +0000,nypost,UK ambassador Peter Mandelson told Jeffrey Eps...,https://nypost.com/2025/09/10/us-news/uk-ambas...
288,71,2025-09-10-20-37-00 +0000,wsj,Nearly half a day after NATO ordered jet fight...,https://www.wsj.com/politics/national-security...
366,68,2025-09-10-17-08-00 +0000,wsj,Former Vice President Kamala Harris said in a ...,https://www.wsj.com/politics/elections/harris-...
25,52,2025-09-11-15-04-29 +0000,nyt,Shooting Suspect Is Dead After Attack at Color...,https://www.nytimes.com/2025/09/10/us/colorado...
386,50,2025-09-10-16-03-48 +0000,nypost,Wholesale inflation unexpectedly fell in Augus...,https://nypost.com/2025/09/10/business/wholesa...
156,50,2025-09-11-09-00-00 +0000,wsj,The killing of the conservative activist Charl...,https://www.wsj.com/politics/political-violenc...
374,48,2025-09-10-16-45-11 +0000,nypost,Qatar calls for retaliation against Israel ‘fr...,https://nypost.com/2025/09/10/world-news/qatar...
137,43,2025-09-11-10-00-00 +0000,latimes,Beloved SoCal octopus will spend her final day...,https://www.latimes.com/california/story/2025-...
290,42,2025-09-10-20-33-00 +0000,wsj,Inside NATO’s Scramble to Shoot Down Russia’s ...,https://www.wsj.com/world/europe/inside-natos-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
